# Mutual information clustering

In [1]:
import sys
sys.path.extend(["../code/src"])

from pathlib import Path
import pandas as pd
import numpy as np
import hdbscan
from Bio.PDB import PDBParser
from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from utils.structure import extract_sequence_from_chain
from utils.alignment import map_seq_to_alignment, is_increasing_monotonic
from utils.commands import call_muscle
from utils.fasta import write_fasta, parse_fasta
from utils.mappings import LEGAL_AAS_THREE, THREE_TO_ONE
# TODO: NEED TO PUT HELPER FUNCTIONS!
from predict_status.helper import map_seqs_to_alignment

def clustering(X, metric="precomputed"):
    """
    """
    clusterer = hdbscan.HDBSCAN(metric=metric,allow_single_cluster=True)
    clusterer.fit(1-X)
    labels = clusterer.labels_

    return clusterer, labels

def find_gap_positions(aln):
    """
    Find columns in the alignment which contain at least one gap
    """
    gap_positions = set()
    for aln_seq in aln.values():
        for idx, res in enumerate(aln_seq):
            if res == '-':
                gap_positions.add(idx)
    gap_positions = sorted(list(gap_positions))
    return gap_positions

# Map clusters to sequences
def map_aln_seq_to_lmi_idxs(aln_seq, gap_positions):
    """
    Map positions in the alignment to rows/columns in the LMI matrix
    Basically, it just skips positions corresponding to gaps
    """
    i = 0
    aln_seq_to_lmi_idx = {}
    for idx, res in enumerate(aln_seq):
        if idx in gap_positions:
            continue
        else:
            aln_seq_to_lmi_idx[idx] = i
            i += 1
    return aln_seq_to_lmi_idx

# Map sequences to PDB indexes
def map_seq_to_pdb_idxs(orig_seq, chain, aa_mapping=THREE_TO_ONE):
    seq_to_pdb = {}
    chain = list(chain) # Avoid indexing problems with HETATMs
    for i, res_i in enumerate(orig_seq):
        for j, res_j in enumerate(chain):
            pdb_idx = res_j._id[1]
            try:
                res_j_mapped = aa_mapping[res_j.resname]
            except KeyError:
                continue
        
            if res_i == res_j_mapped and pdb_idx not in seq_to_pdb.values():
                #print("yay!")
                seq_to_pdb[i] = pdb_idx
                break
    
    # Sanity checks
    vals = seq_to_pdb.values()

    # Check that the list of values increases monotonically
    assert is_increasing_monotonic(list(vals))
    # Check that there are no duplicates in the values
    assert sorted(list(set(vals))) == list(vals)
    return seq_to_pdb


def map_pdb_seq_to_cluster(seq_map, seq_to_pdb, cluster_labels, aln_seq_to_lmi_idx, 
                           gap_positions):
    """
    Map positions in the PDB structure to cluster labels
    """
    pdb_seq_idx_to_cluster = {}
    for orig_idx, aln_idx in seq_map.items():
        if aln_idx not in gap_positions:
            pdb_seq_idx = seq_to_pdb[orig_idx]
            lmi_idx = aln_seq_to_lmi_idx[aln_idx]
            cluster = cluster_labels[lmi_idx]
            pdb_seq_idx_to_cluster[pdb_seq_idx] = cluster
    
    return pdb_seq_idx_to_cluster

def make_pymol_selection_string(residues, chain):
  """
  Create a ready-made string that can be used to select residues in PyMol

  Arguments
  ---------
  residues: np.array of integers, residue indexes to be selected
  chain: str, PDB chain identifier

  Returns
  -------
  selection_string: str
  """
  if len(residues) > 0:
    for idx, residue in np.ndenumerate(residues):
      #print(idx, residue)
      if idx[0] == 0:
        cumulative_string = f"resi {residue}"
      else:
        cumulative_string = "".join([cumulative_string, " or", f" resi {residue}"])

    selection_string = f"select ({cumulative_string}) and chain {chain}"
    print(selection_string)
  else:
    warn("No residues, returning empty string",RuntimeWarning)
    selection_string = ""

def plot_heatmap(lmi_diffs, title, out_path):
    sns.set_context('poster')
    plt.figure(figsize=(8,8))
    ax = sns.heatmap(lmi_diffs,cmap='RdBu_r',vmin=-1,vmax=1, 
                     cbar_kws={'label': 'Δ Mutual information'})
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(left=False, bottom=False)

    plt.xlabel('Residue index')
    plt.ylabel('Residue index')
    plt.suptitle(title)
    plt.savefig(out_path / "lmi_diffs.pdf",dpi=120,bbox_inches='tight')
    plt.clf()
    
out_path = Path("lmi_clustering")
out_path.mkdir(exist_ok=True)
chains_path = Path("../data/processed/pdb_pairs/chains_by_protein/")
annot_path = Path("../data/processed/pdb_pairs/chains_by_protein/annotation_per_psite/")
nma_results_path = Path("../results/new_nma_analysis_per_psite")


ModuleNotFoundError: No module named 'predict_status'

## C-ets-1

In [3]:
psite_cets1, name_cets1 = "P27577_41", "C-ets-1"

uniprot_id, _ = psite_cets1.split("_")
lmi_diffs_cets1 = pd.read_csv(f"../results/new_nma_per_psite_processed/{psite_cets1}/lmi_diffs.csv",index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../results/new_nma_per_psite_processed/P27577_41/lmi_diffs.csv'